In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from time import time

In [2]:
filename = r'Global_Mobility_Report.csv'
mobility_wide = pd.read_csv(
    filename
)

C:\Users\USANCOL3\AppData\Local\Continuum\anaconda3\envs\analysis\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
country_code = 'US'
mobility_wide = mobility_wide.loc[
    mobility_wide.country_region_code == country_code
]

In [4]:
mobility_wide.columns

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')

In [5]:
mobility_wide = mobility_wide.rename(columns={
    'retail_and_recreation_percent_change_from_baseline': 'retail_recreation',
    'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_pharmacy',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline': 'transit_stations',
    'workplaces_percent_change_from_baseline': 'workplaces',
    'residential_percent_change_from_baseline': 'residential'
})

In [ ]:
mobility_wide.dtypes

In [6]:
mobility_wide.memory_usage(deep=True) / 1024**2

Index                   5.133125
country_region_code    37.856799
country_region         44.914846
sub_region_1           41.776830
sub_region_2           44.994155
metro_area             20.532501
iso_3166_2_code        20.923546
census_fips_code        5.133125
date                   42.989924
retail_recreation       5.133125
grocery_pharmacy        5.133125
parks                   5.133125
transit_stations        5.133125
workplaces              5.133125
residential             5.133125
dtype: float64

In [12]:
mobility_wide.isna().sum()

country_region_code         0
country_region              0
sub_region_1              268
sub_region_2            13936
metro_area             672809
iso_3166_2_code        659141
census_fips_code        13668
date                        0
retail_recreation      220784
grocery_pharmacy       260832
parks                  517102
transit_stations       418128
workplaces              22763
residential            324768
dtype: int64

In [ ]:
mobility_wide.head()

In [40]:
def memory_usage(df, detail=False):
    mem = df.memory_usage(deep=True) / 1024**2
    tot_mem = round(mem.sum(), 2)
    print(f"total memory MB: {tot_mem}")
    if detail:
        print(f"\n{mem}")

In [41]:
memory_usage(mobility_wide, detail=True)

total memory MB: 295.05

Index                   5.133125
country_region_code    37.856799
country_region         44.914846
sub_region_1           41.776830
sub_region_2           44.994155
metro_area             20.532501
iso_3166_2_code        20.923546
census_fips_code        5.133125
date                   42.989924
retail_recreation       5.133125
grocery_pharmacy        5.133125
parks                   5.133125
transit_stations        5.133125
workplaces              5.133125
residential             5.133125
dtype: float64


In [ ]:
mobility_wide.date = pd.to_datetime(mobility_wide.date)

In [42]:
mobility_long = mobility_wide.melt(id_vars=[
    'country_region',
    'sub_region_1',
    'sub_region_2',
    'metro_area',
    'census_fips_code',
    'date'
    ], 
    value_vars=[
    'retail_recreation',
    'grocery_pharmacy',
    'parks',
    'transit_stations',
    'workplaces',
    'residential'
    ],
    var_name='activity',
    value_name='percent_change'
)

In [43]:
memory_usage(mobility_long, detail=True)

total memory MB: 1500.41

Index                 0.000122
country_region      269.489079
sub_region_1        250.660978
sub_region_2        269.964930
metro_area          123.195007
census_fips_code     30.798752
date                257.939547
activity            267.564157
percent_change       30.798752
dtype: float64


In [52]:
# write to file
start = time()
mobility_long.to_csv(
    f'{country_code}_mobility_long.csv.gz',
    compression='gzip',
    index=False
)
end = time()
print(end - start)

39.72965955734253


In [59]:
start = time()
us_long = pd.read_csv('./US_mobility_long.zip', compression='zip', parse_dates=['date'])
end = time()
print(end - start)
us_long.head()

6.342523097991943


,country_region,sub_region_1,sub_region_2,metro_area,census_fips_code,date,activity,percent_change
0,United States,NaN,NaN,NaN,NaN,2020-02-15,retail_recreation,6.0
1,United States,NaN,NaN,NaN,NaN,2020-02-16,retail_recreation,7.0
2,United States,NaN,NaN,NaN,NaN,2020-02-17,retail_recreation,6.0
3,United States,NaN,NaN,NaN,NaN,2020-02-18,retail_recreation,0.0
4,United States,NaN,NaN,NaN,NaN,2020-02-19,retail_recreation,2.0


In [60]:
us_long.dtypes

country_region              object
sub_region_1                object
sub_region_2                object
metro_area                 float64
census_fips_code           float64
date                datetime64[ns]
activity                    object
percent_change             float64
dtype: object

In [ ]:
us = mobility_long.loc[
    (mobility_long.country_region == 'United States') &
    (mobility_long.census_fips_code.notna())
]

In [ ]:
us.head()

In [ ]:
us.loc[
    (us.sub_region_1 == 'Alabama') &
    (us.sub_region_2.isin(['Autauga County', 'Baldwin County'])),
    ['date']
].value_counts()

In [ ]:
co.head()

In [ ]:
co.shape

In [ ]:
boulder = co.loc[
    co.sub_region_2 == 'Boulder County'
]

In [ ]:
boulder.columns

In [ ]:
boulder.activity.unique()

In [ ]:
boulder_transit = boulder.loc[
    boulder.activity == 'retail_recreation',
    [
        'date',
        'percent_change'
    ]
]

In [ ]:
# function for shading weekend days in plots
def highlight_weekends(ax, timeseries):
    # access datetime attributes of timeseries
    dt = timeseries.dt
    # create df of weekend start and end dates
    ranges = timeseries[dt.dayofweek >= 5].groupby([dt.year, dt.isocalendar().week]).agg(['min', 'max'])
    # fill area between weekend dates
    for i, tmin, tmax in ranges.itertuples():
        ax.axvspan(tmin, tmax, facecolor='gray', edgecolor='none', alpha=0.1)

In [ ]:
fig, ax = plt.subplots(figsize=(23, 6))
sns.lineplot(x=co.date.unique(), y=boulder_transit.percent_change)
highlight_weekends(ax, co.date)
# ax = sns.lineplot(x=co.date.unique(), y=boulder_transit.percent_change.rolling(7).mean())
# start, end = ax.get_xlim()
# ax.xaxis.set_ticks(np.arange(start + 9, end - 4, 7))
# plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
ax.xaxis.set_tick_params(length=0)
ax.yaxis.set_tick_params(length=0)
plt.xlabel("year-month")
plt.title("transit")
sns.despine(left=True, bottom=True)
plt.show()